In [1]:
import csv 
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras import metrics
from keras.layers import Dense, Activation

/Users/sumanthnukala/Library/Python/3.8/lib/python/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/sumanthnukala/Library/Python/3.8/lib/python/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/sumanthnukala/Library/Python/3.8/lib/python/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/sumanthnukal

# Number (Ind1) (Protein1) (Ind2) (Protein2) (Interacting/not-interacting)

In [2]:
Index = []
Protein_Index = []
Protein = []
result = []
#positive pairs-3899
#negative pairs-1882

In [3]:
file = 'Dataset/Protein_data.txt'
file = ['Dataset/positive.txt','/Users/sumanthnukala/Desktop/Acad/Sem-4/DL/Project/Code/Dataset/negative.txt']

for file1 in file:
    with open(file1) as f: 
        for line in f:
            if ' ' in line:
                Index.append(line.split(' ')[0])
            elif '>' in line:
                temp = line.split('>')[-1].replace('\n','')
                Protein_Index.append(temp)    
            else:
                temp1 = line.replace('\n','')
                Protein.append(temp1)
print('Proteins')
print(len(Protein))
print(len(Index))

Proteins
11562
5781


In [4]:
#Combining Protein Pairs
from operator import add
P1 = []
P2 = []
for i in range(0,len(Protein)):
    if i%2 == 0:
        P1.append(Protein[i])
    elif i%2 == 1:
        P2.append(Protein[i])

protein_pairs = []
protein_pairs = list(map(add, P1, P2))

Protein = protein_pairs

In [5]:
Clusters = {1:['A','G','V'],2:['I','L','F','P'],3:['Y','M','T','S'],4:['H','N','Q','W'],5:['R','K'],6:['D','E'],7:['C']}
Cluster_Indexes={
    'X':1,
    'U':1,
    'A':1,
    'G':1,
    'V':1,
    'I':2,
    'L':2,
    'F':2,
    'P':2,
    'Y':3,
    'M':3,
    'T':3,
    'S':3,
    'H':4,
    'N':4,
    'Q':4,
    'W':4,
    'R':5,
    'K':5,
    'D':6,
    'E':6,
    'C':7
}

# Conjoint triad method

In [6]:
#example
seq = 'MREIVHIQAG'
print(seq)
for index,value in Cluster_Indexes.items():
    seq = seq.replace(index,str(value))

print(seq)

MREIVHIQAG
3562142411


In [7]:
#Encoding each amino acid in the protein with the corresponding cluster vaue from Cluster_Indexes
Encoded_protein = []
for seq in Protein:
    temp = seq
    for index,value in Cluster_Indexes.items():
        temp = temp.replace(index,str(value))
    Encoded_protein.append(temp)
    temp = ''

#print('Encoded_proteins')
#print(len(Encoded_protein))
#print(Encoded_protein[0])

In [8]:
#creating a vector with permutations of the cluster indexes.
standard = [x for x in range(111, 778) if '8' not in str(x) and '9' not in str(x) and '0' not in str(x)]
print(standard)

[111, 112, 113, 114, 115, 116, 117, 121, 122, 123, 124, 125, 126, 127, 131, 132, 133, 134, 135, 136, 137, 141, 142, 143, 144, 145, 146, 147, 151, 152, 153, 154, 155, 156, 157, 161, 162, 163, 164, 165, 166, 167, 171, 172, 173, 174, 175, 176, 177, 211, 212, 213, 214, 215, 216, 217, 221, 222, 223, 224, 225, 226, 227, 231, 232, 233, 234, 235, 236, 237, 241, 242, 243, 244, 245, 246, 247, 251, 252, 253, 254, 255, 256, 257, 261, 262, 263, 264, 265, 266, 267, 271, 272, 273, 274, 275, 276, 277, 311, 312, 313, 314, 315, 316, 317, 321, 322, 323, 324, 325, 326, 327, 331, 332, 333, 334, 335, 336, 337, 341, 342, 343, 344, 345, 346, 347, 351, 352, 353, 354, 355, 356, 357, 361, 362, 363, 364, 365, 366, 367, 371, 372, 373, 374, 375, 376, 377, 411, 412, 413, 414, 415, 416, 417, 421, 422, 423, 424, 425, 426, 427, 431, 432, 433, 434, 435, 436, 437, 441, 442, 443, 444, 445, 446, 447, 451, 452, 453, 454, 455, 456, 457, 461, 462, 463, 464, 465, 466, 467, 471, 472, 473, 474, 475, 476, 477, 511, 512, 513, 514,

In [9]:
#Creating protein vectors
protein_vectors = []
for prt in Encoded_protein:
    b = [prt[i:i+3] for i in range(len(prt)-2)]
    temp = [0]*343
    #temp1 = list(temp)
    for amino in set(b):
        loc = standard.index(int(amino))
        count1 = b.count(amino)
        temp[loc] = count1
    protein_vectors.append(temp)

print(len(protein_vectors))
print(protein_vectors[0])

5781
[7, 8, 4, 6, 10, 10, 2, 10, 17, 7, 8, 8, 9, 2, 11, 15, 6, 4, 4, 5, 1, 7, 11, 7, 1, 1, 4, 1, 5, 14, 2, 7, 5, 1, 1, 2, 14, 8, 7, 5, 4, 0, 1, 1, 3, 2, 0, 0, 0, 15, 12, 10, 10, 4, 6, 0, 14, 22, 17, 10, 12, 10, 1, 13, 10, 8, 8, 9, 13, 3, 9, 13, 6, 5, 10, 3, 3, 13, 8, 4, 9, 11, 8, 1, 11, 5, 10, 3, 3, 8, 0, 3, 1, 5, 0, 1, 1, 0, 10, 15, 10, 4, 7, 10, 1, 9, 10, 15, 8, 11, 10, 1, 12, 8, 17, 4, 7, 5, 0, 7, 6, 2, 1, 3, 2, 0, 7, 4, 5, 4, 4, 5, 1, 5, 8, 5, 2, 2, 9, 2, 1, 1, 1, 1, 0, 0, 0, 5, 8, 7, 5, 4, 8, 0, 8, 12, 8, 8, 10, 1, 3, 4, 6, 6, 3, 4, 2, 0, 3, 3, 2, 2, 5, 2, 0, 2, 5, 4, 7, 3, 2, 0, 4, 6, 4, 0, 1, 1, 0, 0, 0, 1, 1, 2, 0, 0, 4, 10, 8, 4, 4, 4, 3, 8, 10, 6, 8, 6, 6, 4, 5, 8, 6, 0, 1, 2, 0, 5, 13, 5, 6, 2, 2, 0, 4, 9, 4, 4, 2, 3, 1, 2, 6, 7, 1, 3, 1, 0, 0, 0, 0, 1, 2, 0, 1, 5, 6, 7, 4, 3, 2, 1, 6, 14, 10, 7, 6, 4, 0, 8, 12, 10, 2, 5, 3, 0, 4, 3, 2, 1, 2, 3, 0, 4, 7, 2, 1, 2, 1, 0, 4, 8, 6, 2, 2, 3, 1, 1, 1, 1, 0, 0, 0, 0, 1, 2, 0, 0, 3, 0, 0, 2, 1, 1, 0, 1, 0, 0, 4, 4, 0, 0, 0, 3, 0, 2,

In [10]:
print(len(protein_vectors[0]))

343


In [11]:
#Splitting the protein pairs and generating the interaction indexes where the first half of the protein pairs
#are interacting and the second half is non-interacting.
Interaction_index = []
for i in range(0,int(len(protein_vectors))):
    if i < 3899:
        Interaction_index.append(1)
    else:
        Interaction_index.append(0)
        
print(len(Interaction_index))

5781


In [12]:
protein_vectors = np.array(protein_vectors)
print(protein_vectors.shape)

(5781, 343)


In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(protein_vectors, Interaction_index, test_size=0.2, random_state=25)

In [14]:
#defining the function to calculate other metrics
from keras import backend as K
def Model_recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def Model_precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def Model_f1(y_true, y_pred):
    precision = Model_precision(y_true, y_pred)
    recall = Model_recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Deep Autoencoder

In [15]:
from keras.models import Sequential
from keras import metrics
from keras.layers import Dense, Activation
import keras
from keras import layers

In [16]:
input_prt = keras.Input(shape=(343,))
encoded = layers.Dense(128, activation='relu')(input_prt)
encoded = layers.Dense(64, activation='relu')(encoded)
encoded = layers.Dense(32, activation='relu')(encoded)

decoded = layers.Dense(64, activation='relu')(encoded)
decoded = layers.Dense(128, activation='relu')(decoded)
decoded = layers.Dense(343, activation='sigmoid')(decoded)

In [46]:
autoencoder = keras.Model(input_prt, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
autoencoder.summary()
autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 343)]             0         
_________________________________________________________________
dense (Dense)                (None, 128)               44032     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_4 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_5 (Dense)              (None, 343)              

Epoch 45/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 46/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 47/100
19/19 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 48/100
19/19 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 49/100
19/19 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 50/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 51/100
19/19 [==============================] 

Epoch 97/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 98/100
19/19 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 99/100
19/19 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 100/100
19/19 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00


In [28]:
encoded_imgs = autoencoder.predict(protein_vectors)
from sklearn.model_selection import train_test_split
encoded_imgs1 = np.array(encoded_imgs)
print(type(encoded_imgs1))
Interaction_index = np.array(Interaction_index)
x_train, x_test, y_train, y_test = train_test_split(encoded_imgs1, Interaction_index, test_size=0.2, random_state=25)
print(encoded_imgs.shape)

<class 'numpy.ndarray'>
(5781, 343)


In [29]:
#Keras models
# define the keras model
#model1
model1 = Sequential()
model1.add(Dense(343, input_dim=343, activation='relu'))
model1.add(Dense(128, activation='relu'))
model1.add(Dense(64, activation='relu'))
model1.add(Dense(32, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))
# compile the keras model
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [45]:
model1.fit(x_train, y_train, epochs=100, batch_size=10, verbose=1)

Epoch 1/100
463/463 [==============================] - 1s 1ms/step - loss: 0.6331 - accuracy: 0.6719
Epoch 2/100
463/463 [==============================] - 1s 1ms/step - loss: 0.6331 - accuracy: 0.6719
Epoch 3/100
463/463 [==============================] - 1s 1ms/step - loss: 0.6330 - accuracy: 0.6719
Epoch 4/100
463/463 [==============================] - 1s 1ms/step - loss: 0.6329 - accuracy: 0.6719
Epoch 5/100
463/463 [==============================] - 1s 1ms/step - loss: 0.6331 - accuracy: 0.6719
Epoch 6/100
463/463 [==============================] - 1s 1ms/step - loss: 0.6329 - accuracy: 0.6719
Epoch 7/100
463/463 [==============================] - 1s 1ms/step - loss: 0.6330 - accuracy: 0.6719
Epoch 8/100
463/463 [==============================] - 1s 1ms/step - loss: 0.6331 - accuracy: 0.6719
Epoch 9/100
463/463 [==============================] - 1s 1ms/step - loss: 0.6331 - accuracy: 0.6719
Epoch 10/100
463/463 [==============================] - 1s 1ms/step - loss: 0.6330 - accura

In [31]:
#Test Performance
print("Model - 1")
print("---------------------------------------------------------------------------------------------")
print(model1.metrics_names)
print(model1.evaluate(x_test,y_test))

Model - 1
---------------------------------------------------------------------------------------------
['loss', 'accuracy']
37/37 [==============================] - 0s 941us/step - loss: 0.6243 - accuracy: 0.6845
[0.6242917776107788, 0.6845289468765259]


In [36]:
#preparing the testing dataset
import pandas as pd
Pid_file = '119_SCZ_interactions.xls'

dfs = pd.read_excel(Pid_file)

In [37]:
Prt_name = list(dfs['P1_id'])
Prt_uniprot_id = list(dfs['Uniprot_ID'])


In [38]:
import requests as r
from Bio import SeqIO
from io import StringIO
import time

Prt_uniprot_seq = []
for cID in Prt_uniprot_id:
    baseUrl="http://www.uniprot.org/uniprot/"
    currentUrl=baseUrl+cID+".fasta"
    response = r.post(currentUrl)
    time.sleep(1)
    cData=''.join(response.text)
    Seq=StringIO(cData)
    pSeq=SeqIO.parse(Seq,'fasta')
    for i in pSeq:
        print(i.id)
        seq1 = str(i.seq)
        Prt_uniprot_seq.append(seq1)
        print('*******************')

tr|C4P094|C4P094_HUMAN
*******************
sp|Q99689|FEZ1_HUMAN
*******************
sp|Q5SW79|CE170_HUMAN
*******************
sp|Q9UKE5|TNIK_HUMAN
*******************
tr|A0A0C4DG40|A0A0C4DG40_HUMAN
*******************
sp|P63261|ACTG_HUMAN
*******************
sp|P35609|ACTN2_HUMAN
*******************
sp|P51648|AL3A2_HUMAN
*******************
sp|P18848|ATF4_HUMAN
*******************
tr|A8MVZ6|A8MVZ6_HUMAN
*******************
tr|E9PHM6|E9PHM6_HUMAN
*******************
sp|O15442|MPPD1_HUMAN
*******************
sp|Q99459|CDC5L_HUMAN
*******************
sp|P10909|CLUS_HUMAN
*******************
tr|A5PKV2|A5PKV2_HUMAN
*******************
sp|Q14203|DCTN1_HUMAN
*******************
tr|A0A0S2Z3B5|A0A0S2Z3B5_HUMAN
*******************
sp|Q14204|DYHC1_HUMAN
*******************
tr|A0A1C7CYX9|A0A1C7CYX9_HUMAN
*******************
sp|Q14195|DPYL3_HUMAN
*******************
sp|P13639|EF2_HUMAN
*******************
sp|Q92935|EXTL1_HUMAN
*******************
tr|A0A140VJJ8|A0A140VJJ8_HUMAN
*******************
t

In [39]:
test_protein_pairs = []
for i in range(1,len(Prt_uniprot_seq)):
    new_prt = Prt_uniprot_seq[0]+Prt_uniprot_seq[i]
    test_protein_pairs.append(new_prt)

In [40]:
#Encoding each amino acid in the protein with the corresponding cluster vaue from Cluster_Indexes
Encoded_protein_schizophrenia = []
for seq in test_protein_pairs:
    temp = seq
    for index,value in Cluster_Indexes.items():
        temp = temp.replace(index,str(value))
    Encoded_protein_schizophrenia.append(temp)
    temp = ''

In [41]:
#Creating protein vectors
protein_vectors_schizophrenia = []
for prt in Encoded_protein_schizophrenia:
    b = [prt[i:i+3] for i in range(len(prt)-2)]
    temp = [0]*343
    #temp1 = list(temp)
    for amino in set(b):
        loc = standard.index(int(amino))
        count1 = b.count(amino)
        temp[loc] = count1
    protein_vectors_schizophrenia.append(temp)


protein_vectors_schizophrenia = np.array(protein_vectors_schizophrenia)
print(protein_vectors_schizophrenia.shape)

(118, 343)


In [42]:
encoded_imgs1 = autoencoder.predict(protein_vectors_schizophrenia)
results = model1.predict_classes(encoded_imgs1)
print(results.shape)
print(np.count_nonzero(results == 1))
print(np.count_nonzero(results == 0))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
(118, 1)
118
0


In [43]:
model1.save('models/model_ae.h5')